In [1]:
import itertools
import pandas as pd
from tqdm.notebook import tqdm
%run keras_sentinel.ipynb

2024-05-16 17:54:38.979574: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 17:54:39.850089: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

test_year = 2020

train_years = [2017, 2018, 2019]
all_seasons = [3, 6, 9, 12]

season_combinations = itertools.chain.from_iterable(
    itertools.combinations(all_seasons, r) for r in range(1, len(all_seasons)+1))
season_combinations = list(season_combinations)

In [ ]:
results = []
for season_combination in tqdm(season_combinations):
    filepaths = []
    for year in train_years:
        for season in season_combination:
            file_key = f'processed_treesat_{year}*{season}.npy'
            filepaths.extend(sorted(list(Path('seasonal_median').glob(file_key))))
            
    selected_data = []
    for filepath in filepaths:
        with open(filepath, 'rb') as f:
            data = np.load(f)
        selected_data.append(data)
    features = np.stack(selected_data, axis=1)

    model_name = f'conv_seasons_{"_".join(map(str, season_combination))}_years_{"_".join(map(str, train_years))}.keras'
    model_eval = KerasModelCreator().run(labels, features, model_name, model_dir, epochs=10, overwrite=True)
    results.append(model_eval)

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1/10
11336/11336 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - f1_majority: 0.1977 - loss: 0.0217 - r2_score: 0.0604 - root_mean_squared_error: 0.2076
Epoch 2/10
11336/11336 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - f1_majority: 0.3760 - loss: 0.0190 - r2_score: 0.1550 - root_mean_squared_error: 0.1938
Epoch 3/10
11336/11336 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - f1_majority: 0.4143 - loss: 0.0183 - r2_score: 0.1757 - root_mean_squared_error: 0.1901
Epoch 4/10
11336/11336 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - f1_majority: 0.4259 - loss: 0.0181 - r2_score: 0.1863 - root_mean_squared_error: 0.1889
Epoch 5/10
11336/11336 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - f1_majority: 0.4373 - loss: 0.0178 - r2_score: 0.1952 - root_mean_squared_error: 0.1873
Epoch 6/10
11336/11336 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - f1_majority: 0.4527 - loss: 0.0176 - r2_score: 0.2044 - root_mean_squared_error: 0.1860
Epoch 7/10
11336/11336 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - f1_majority: 0.4652 - loss: 0.0173 - r2_score: 0.2120 - roo

In [ ]:
results_df = pd.DataFrame(results)
results_df.insert(0, 'season_comb', season_combinations)
results_df

In [ ]:
results = []
for season_combination in tqdm(season_combinations):
    if len(season_combination) < 3:
        continue
    filepaths = []
    selected_data = []
    for season in season_combination:
        filepath = Path('seasonal_median').joinpath(f'processed_treesat_{test_year}{str(season).zfill(2)}.npy')
        with open(filepath, 'rb') as f:
            data = np.load(f)
        selected_data.append(data)
    features = np.stack(selected_data, axis=1)

    X1, X2, y1, y2 = KerasModelCreator().split_and_normalise(
        labels, features
    )
    
    model_name = f'conv_seasons_{"_".join(map(str, season_combination))}_years_{"_".join(map(str, train_years))}.keras'
    model = tf.keras.models.load_model(model_dir.joinpath(model_name))

    model_eval = model.evaluate(
        np.concatenate((X1, X2)), np.concatenate((y1, y2)), verbose=0, return_dict=True
    )
    
    results.append(model_eval)

In [ ]:
# major_pred = y_pred.argmax(axis=1)
# major_true = y_true.argmax(axis=1)

# cr = classification_report(
#     major_true, major_pred, target_names=df[target].columns, 
#     output_dict=True, zero_division=np.nan
# )
# df_report = pd.DataFrame(cr).T
# df_report['support'] = df_report['support'].astype(int)

# display(HTML(df_report.round(decimals=2).to_html()))